In [2]:
!pip install factor_analyzer

     |████████████████████████████████| 41 kB 306 kB/s 
  Created wheel for factor-analyzer: filename=factor_analyzer-0.4.0-py3-none-any.whl size=41455 sha256=cac35ea1602d74232f44c9f12e7fc79a4a966ea6e60d84f4e68cd91302fe0294
  Stored in directory: /root/.cache/pip/wheels/ac/00/37/1f0e8a5039f9e9f207c4405bbce0796f07701eb377bfc6cc76
Successfully built factor-analyzer


In [6]:
!pip install scikit-learn-extra


     |████████████████████████████████| 1.7 MB 5.0 MB/s 


In [7]:
# importing packaged
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import os
# !pip install factor_analyzer
import factor_analyzer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from scipy.stats import zscore
from sklearn import cluster
from sklearn.metrics import silhouette_score,davies_bouldin_score,calinski_harabasz_score
from sklearn import preprocessing

from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

#Ignore warnings
import warnings
# warnings.filterwarnings("ignore")
import numpy as np
from datetime import datetime
from collections import Counter
from collections import defaultdict
from tqdm import tqdm
from random import choices
from sklearn.cluster import DBSCAN
from sklearn import cluster
from sklearn_extra.cluster import KMedoids

In [10]:
os.getcwd() 

'/content'

In [22]:
!ls

sample_data


## Read file and generating EDA

In [13]:
# read file
df=pd.read_csv("/processed_file.csv")
df=df.iloc[:,1:]
# converting date to datime
# df["date"]=df["date"].apply(lambda x:datetime.strptime(str(x),"%Y%m%d"))
df.head()

FileNotFoundError: ignored

In [ ]:
# Creating EDA table and adding multilevel index to dimensions
df_dim=df.iloc[0:1,].T
# create major index
idx=[x.split(".")[0] for x in list(df_dim.index)]
df_dim.columns=["idx"]
df_dim["idx"]=idx
df_dim.head()

In [ ]:
# adding EDA columns
cols=list(df.columns)
df_dim["type"]=df.apply(lambda x:np.dtype(x),axis=0)
df_dim["Unique_ct"]=df.apply(lambda x:len(x.unique()),axis=0)
df_dim["ct_nan"]=df.apply(lambda x:x.isna().sum(),axis=0)
df_dim.head()

In [ ]:
## adding key metrics
df_dim["Counter"]=None
df_dim["min"]=None
df_dim["max"]=None
df_dim["mean"]=None
# df_dim["outlier_ct"]=None

for element in list(df_dim.index):
    if df_dim.loc[element,"Unique_ct"]>1:   
        
        df_dim.at[element,"Counter"]=Counter(df[element]).most_common(5)
    
        if df_dim.loc[element,"type"]!="object":
            df_dim.at[element,"Counter"]=Counter(df[element]).most_common(5)
            df_dim.at[element,"min"]=np.nanmin(df[element])
            df_dim.at[element,"max"]=np.nanmax(df[element])
            df_dim.at[element,"mean"]=np.nanmean(df[element])
            # df_dim.at[element,"outlier_ct"]=df[element].apply(lambda x:outliers(x,element))
    else:
        continue
   


In [ ]:
# calculating key metrics for dates
df["date"]=df["date"].apply(lambda x:datetime.strptime(str(x),"%Y%m%d"))
df_dim.at["date","min"]=np.min(df["date"]) # .apply(lambda x:datetime.strptime(str(x),"%Y%m%d")))
df_dim.at["date","max"]=np.max(df["date"]) # .apply(lambda x:datetime.strptime(str(x),"%Y%m%d")))
df_dim["spread"]=None
df_dim.at["date","spread"]=df_dim.at["date","max"]-df_dim.at["date","min"]

In [ ]:
df_dim.head()

In [ ]:
# exporting object to excel
df_dim.to_excel('dimensions.xlsx')

# Testing categorical variable encoding

In [ ]:
# encoding country
df_dim.loc["geoNetwork.country",]

In [ ]:
# methods available
# target: value of the target classifier
# target_mean: mean of the value of the target
# counter: counter of the class

In [ ]:
# generating a copy of the data set for testing
df_copy=df.copy()
methods=["counter","target","one_hot"]

In [ ]:

col="geoNetwork.country"

def categorical_transfo(df_copy,col,method,liste_new_features):
    
    if method=="target":
        # randomly generating target labels
        numbers=[0,1]
        weights = [0.5,0.5]
        df_copy["target"] = choices(numbers, weights, k=len(df))
        df_copy[col+"_target"] =df_copy["target"]
        liste_new_features.append("target")
        liste_new_features.append(col+"_target")
        
    if method=="counter":
        dico=dict(Counter(df_copy[col]))
        df_copy[col+"_counter"]=df_copy[col].apply(lambda x: dico[x])
        liste_new_features.append(col+"_counter")
        
    if method=="one_hot":
        data=pd.get_dummies(df_copy[col], prefix=None, prefix_sep='_',drop_first=False,dummy_na=True)
        return data, list(data.columns)
        
        
    return df_copy,liste_new_features

liste_new_features=[]
for method in methods:    
    retour,liste_new_features=categorical_transfo(df_copy,col,method,liste_new_features)

In [ ]:
set1=set(list(df_copy[col].unique()))
set2=set(list(retour.columns))
print(len(set1),len(set2))
print(set2-set1)

## Preparing RFM data set

In [ ]:
# calculating RFM, frequency of visits in the last 121 days
dico=dict(Counter(df["fullVisitorId"]))
df["Frequency"]=df["fullVisitorId"].apply(lambda x: dico[x])

In [ ]:
df.sort_values(by="Frequency", ascending=False,inplace=True)
df["Frequency"].head(2)

In [ ]:
# printing key date indicators
maxi_date=np.max(df["date"])
mini_date=np.min(df["date"])
spread=(maxi_date-mini_date)
print(maxi_date,mini_date,spread)

In [ ]:
# calculating inverse of recency, total spread - time since last visit in days.
df["Recency"]=(df["date"]-(pd.to_datetime(np.min(df["date"])))).dt.days
df[["date","Recency"]].head(5)

In [ ]:
# generating a random monetary number for running RFM as data is not available
rd=np.random.choice(len(df), 3)
numbers=[0,1]
weights = [0.97,0.03]
million_samples = choices(numbers, weights, k=len(df))
sample=np.random.randint(low=50, high=1000, size=len(df))
liste=sample*million_samples
df["Monetary"]=liste


In [ ]:
# generating RFM table
rfm=df.groupby("fullVisitorId").agg({"Recency":"min", "Monetary":"sum", "Frequency":"min"})
rfm.head(2)

In [ ]:
## performing EDA
rfm.describe().round(1)

In [ ]:
# EDA - visualizing key customers data in a violin plot
cols=list(rfm.columns)
units=["days","$","times"]
ax=["ax1","ax2","ax3"]
liste_zip=zip(cols,units,ax)

# Ploting the figure
fig = plt.figure()
fig.set_size_inches(14, 6)

i=1
for metrics,units,ax in liste_zip:
    ax = plt.subplot(1,4,i)
    ax = sns.violinplot(data=np.array(rfm[metrics]),palette="Set3")
    # ax = sns.violinplot(x=cols, y=metrics, data=Det1,palette="Set3")
    ax.set_ylabel('{} in {}'.format(metrics,units),fontsize=14)
    ax.set_title("{} By Cluster - {}".format(metrics,units),fontsize=14)
    i+=1

plt.tight_layout()
plt.show()

In [ ]:
# Returning data frame and dictionnary with respective labels and counts of outliers.
def outliers(df_out):
    df_out=df_out.copy()
    dico_out={}
    
    for col in list(df_out.columns):
        Q1 = df_out[col].quantile(0.25)
        Q3 = df_out[col].quantile(0.75)
        IQR = Q3 - Q1
        mask=(df_out[col] < (Q1 - 1.5 * IQR)) | (df_out[col] > (Q3 + 1.5 * IQR))
        df_out["out_"+col]=np.int64(mask)
    
        # returning number of outliers
        out_number=df_out["out_"+col].sum()
        dico_out["out_"+col]=out_number
    
    return df_out, dico_out

df_out,dico_out=outliers(rfm)
dico_out

In [ ]:
# scaling and unscaling handy functions, returns either a DataFrame or a numpy array
def unscale(df_scaled,scaler,array=False):
    if not array:        
        df_unscaled=pd.DataFrame(scaler.inverse_transform(df_scaled),index=df_scaled.index, columns=list(df_scaled.columns))
    else:
        df_unscaled=scaler.inverse_transform(df_scaled)
    return df_unscaled

# takes in an INDEXED data frame and returns a SCALED data frame AND its scaler for future reverse scaling
def scale(df,array=False): 
    cols=list(df.columns)        
    scaler = preprocessing.StandardScaler().fit(df)
    index=df.index
    
    if array:
        df_scaled=scaler.transform(df)
    else:     
        df_scaled=pd.DataFrame(scaler.transform(df),index=index, columns=cols)

    return df_scaled,scaler,index,cols
                           
rfm_scaled, rfm_scaler,index,cols=scale(rfm,array=True)                           
# unscale(rfm_scaled,rfm_scaler)
rfm_scaled

In [ ]:
# returns the pca model and cumulative pca variation for visualization purpose
def fit_PCA(X_scaled):
    pca1 = PCA().fit(X_scaled)
    liste_pca=list(np.cumsum(pca1.explained_variance_ratio_))
    liste_pca.append(0)
    liste_pca=sorted(liste_pca)
    pca_cum=pd.Series(liste_pca, index=range(0,len(liste_pca)),name="pca_cum").round(2)
    
    return pca1,pca_cum

# visualize PCA
def spree_PCA(pca_cum):
    fig = plt.figure()
    fig.set_size_inches(25, 6)

    # Visualizing PCA
    x_values2=np.arange(0,5)
    ax0 = plt.subplot(121)
    ax0=pca_cum.plot(marker='o')
    ax0.set_xlabel('Number of components',size=12)
    ax0.set_ylabel('Cumulative explained variance',size=12)
    ax0.set_title("PCA Analysis", fontsize=14, fontweight='bold',pad=10)
    ax0.axvline(x=2,color='black',alpha=0.5,ls="--")
    for i, txt in enumerate(pca_cum):
        ax0.annotate(f'{round(txt,2)*100}%', (x_values2[i]-0.07,pca_cum[i]+0.02))
    plt.show()
    
 
pca1,pca_cum=fit_PCA(rfm_scaled)
spree_PCA(pca_cum)


In [ ]:
## transforming data with PCA=2

cols=list(rfm.columns)
index=rfm.index
def pca_transfo(X_scaled, cols,index,pcn=2, array=True):
    col_pca=[]
    pca2=PCA(n_components=pcn).fit(X_scaled)
    pca_transformed=pca2.transform(X_scaled)
    pca_components=pd.DataFrame(pca2.components_.transpose(),index=cols)
    pca_components.columns=['PC1','PC2']
    
    if not array:
        pca_transformed=pd.DataFrame(data=pca_transformed,columns=['PC1','PC2'], index=index)
        
    return pca_transformed,pca_components

pca_transformed,pca_components=pca_transfo(rfm_scaled, cols,index,pcn=2, array=False)   
pca_transformed

In [ ]:
pca_transformed.columns

In [ ]:
## performing k-means clustering
# Bouldin The minimum score is zero, with lower values indicating better clustering.
# *Higher* is better for the Calinski-Harabasz index: means better-defined clusters.
# WCSS is defined as the sum of the squared distance between each member of the cluster and its centroid.
maxi_pca=2

def cluster_predict(df,end,mod="kmeans"):
    # uses cache, tries to download file
    try:
        scores=pd.read_csv("scores_{}.csv".format(mod),index_col="index")
        return scores
    except:           
        scores=defaultdict(list)
        
    start=2
    
    try:
        del(df["CLUSTERS"])
    except:
        pass
    
    df=np.array(df)

    for n in range(start,end):
        if mod=="kmeans":            
            model=cluster.KMeans(n_clusters=n,random_state=10, max_iter=300)
        if mod=="dbscan":
            model=DBSCAN(eps=n, min_samples=2)            
        
        model.fit(df)      
        labels = model.labels_
        
        try:
            scores["inertia"].append(np.int64(model.inertia_))
        except:
            pass

        scores["silhouette"].append(round(silhouette_score(df,labels),2))
        scores["bouldin"].append(round(davies_bouldin_score(df,labels),2))  
        scores["calinski"].append(np.int64(calinski_harabasz_score(df,labels)))
        
    scores=pd.DataFrame(data=scores, index=range(start,end)) 
    
    # UPLOAD
    (cluster_scores.reset_index()).to_csv("scores_{}.csv".format(mod),index=False)
    
    return scores

def plot_elbow(cluster_scores):
    plt.plot(range(2, 7), cluster_scores["inertia"].values)
    plt.title('Elbow Method')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.show()

cluster_scores=cluster_predict(pca_transformed,end=7,mod="kmeans")
cluster_scores


In [ ]:
plot_elbow(cluster_scores)

In [ ]:
# set the number of clusters
################
n_clusters=5
############

In [ ]:
## performing k-means clustering on df and returns labels you need to set the number of clusters
## as well as all the needed groupby dataframes

def cluster_predict(n, pca_df,df_scaled, df, index, cols):
    cluster_names=["A","B","C","D","E","F","G"]
    cluster1="CLUSTERS"
    index=df.index
    
    try:
        del(df["CLUSTERS"])
    except:
        pass
    try:
        del(pca_df["CLUSTERS"])
    except:
        pass
    model=cluster.KMeans(n_clusters=n,random_state=10)
    labels=list(model.fit_predict(pca_df))
    
    try:        
        df_scaled[cluster1]=labels
    except:
        df_scaled=pd.DataFrame(data=df_scaled, columns=cols,index=index)
        df_scaled[cluster1]=labels
    unique_labels=list(np.unique(labels))
    
    df_scaled[cluster1]=df_scaled[cluster1].replace(unique_labels,cluster_names[:len(unique_labels)])
    
    df[cluster1]=df_scaled[cluster1]
    
    
    groupe=df.groupby("CLUSTERS").mean().round(2)
    groupe["count"]=df.groupby("CLUSTERS").size().round(2)
    
    groupe_z=df_scaled.groupby("CLUSTERS").mean().round(2)
    pca_df[cluster1]=df_scaled[cluster1]
    
    return pca_df,df,groupe_z, groupe
    
pca_df,df_clusters,groupe_z, groupe=cluster_predict(n_clusters, pca_transformed,rfm_scaled, rfm,index,cols)

pca_df.head()

In [ ]:
groupe

In [ ]:
groupe_z

In [ ]:
pca_cum

In [ ]:
def visualization_1(data,pca_cum,groupe_z,n):
    HUE="CLUSTERS"
    pcn=2
    
    fig = plt.figure()
    fig.set_size_inches(14, 5)

    ax1 = plt.subplot(121)
    ax1=sns.scatterplot(data=data, x="PC1", y="PC2", hue=HUE, palette="deep",style=HUE)
    ax1.set_xlabel("Principal Component 1",fontsize=12)
    ax1.set_ylabel("Principal Component 2",fontsize=12)
    ax1.set_title("Customer Segmentation \n with PCA={} ({} of var) and KMeans (k={})".format(pcn,pca_cum[pcn],n),fontsize=14)
    ax1.set_facecolor('#eafff5')
    handles, labels = ax1.get_legend_handles_labels()
    # sort both labels and handles by labels
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
    ax1.legend(handles, labels)

    ax2 = plt.subplot(122)
    ax2 = sns.heatmap(groupe_z,vmin=-.3, vmax=1, cmap=sns.diverging_palette(240, 20, n=9),center=0,annot=True)
    ax2.set_title("Interpretation: \nDimension Weight",fontsize=14)
    ax2.set_xticklabels(cols, fontdict=None, minor=False,rotation=45)
    ax2.set_ylabel(HUE,fontsize=12)
  
    plt.show()
        

In [ ]:
visualization_1(pca_df,pca_cum,groupe_z,n=n_clusters)

# Performing DBSCAN and Kmedoids clustering

In [ ]:
# from sklearn import cluster
# from sklearn_extra.cluster import KMedoids
arr=np.array(pca_transformed)
dbscan = cluster.DBSCAN(eps=225, min_samples=20)
clustering_labels = dbscan.fit_predict(arr)

In [ ]:
model=KMedoids(n_clusters=4, random_state=0).fit(arr)
clustering_labels=model.labels_

In [ ]:
np.unique(clustering_labels)

In [ ]:
arr=np.array(rfm_scaled)
model=KMedoids(n_clusters=4, random_state=0).fit(arr)
clustering_labels=model.labels_

In [ ]:
np.unique(clustering_labels)

In [ ]:
dbscan = cluster.DBSCAN(eps=225, min_samples=20)
clustering_labels = dbscan.fit_predict(arr)

In [ ]:
np.unique(clustering_labels)